# Fine Tuning Flan T5

References:

1. https://www.datacamp.com/tutorial/flan-t5-tutorial
2. https://www.youtube.com/watch?v=r6XY80Z9eSA (Ignore Lightning, this also uses trainer instead of seq2seq trainer)
3. https://discuss.huggingface.co/t/trainer-vs-seq2seqtrainer/3145 (Why I'm using seq2seq trainer instead of regular trainer)

In [1]:
!nvidia-smi

Tue Jul 16 21:12:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:2B:00.0  On |                  N/A |
|  0%   35C    P0             18W /  285W |     555MiB /  16376MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import gc

gc.collect()

if torch.cuda.is_available():
    print("CUDA is available. Device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
    torch.cuda.empty_cache() # Empty it first otherwise I get the dreaded CUDA out of memory
else:
    print("CUDA is not available.")

CUDA is available. Device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4070 Ti SUPER


In [3]:
DATASET_FILE_PATH= 'temp/zillow_qa_dataset.json'

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import nltk
import evaluate
import numpy as np
from datasets import load_dataset

dataset = load_dataset('json', data_files=DATASET_FILE_PATH)

dataset = dataset['train'].train_test_split(test_size=0.2)

dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 561
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 141
    })
})

In [5]:
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    targets = [a for a in examples['answer']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

tokenized_dataset

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/561 [00:00<?, ? examples/s]

c:\Users\kevin\Projects\mads-capstone-project\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/141 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 561
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 141
    })
})

In [6]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def extract_number(text):
    import re
    match = re.search(r'\$\d+', text)
    if match:
        return int(match.group()[1:])
    return None

In [7]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    y_true = [extract_number(label) for label in decoded_labels]
    y_pred = [extract_number(pred) for pred in decoded_preds]
    
    y_true = [val for val in y_true if val is not None]
    y_pred = [val for val in y_pred if val is not None]

    if y_true and y_pred:
        mape = mean_absolute_percentage_error(y_true, y_pred)
    else:
        mape = None

    result['mape'] = mape
    return result

In [8]:
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

In [9]:
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)


trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)


trainer.train()


c:\Users\kevin\Projects\mads-capstone-project\.venv\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/213 [00:00<?, ?it/s]

c:\Users\kevin\Projects\mads-capstone-project\.venv\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.49760690331459045, 'eval_rouge1': 0.8941614728848757, 'eval_rouge2': 0.8811465721040185, 'eval_rougeL': 0.8940755068414631, 'eval_rougeLsum': 0.8941614728848757, 'eval_mape': 33.42275119596515, 'eval_runtime': 28.3294, 'eval_samples_per_second': 4.977, 'eval_steps_per_second': 1.271, 'epoch': 1.0}


...

{'eval_loss': 0.47702109813690186, 'eval_rouge1': 0.8940038684719521, 'eval_rouge2': 0.881166272655634, 'eval_rougeL': 0.8940325238197568, 'eval_rougeLsum': 0.8939823769610992, 'eval_mape': 28.82100022324944, 'eval_runtime': 28.8902, 'eval_samples_per_second': 4.881, 'eval_steps_per_second': 1.246, 'epoch': 2.0}


c:\Users\kevin\Projects\mads-capstone-project\.venv\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.46810510754585266, 'eval_rouge1': 0.8934486711082443, 'eval_rouge2': 0.8804669030732856, 'eval_rougeL': 0.8934880722114752, 'eval_rougeLsum': 0.8934952360484264, 'eval_mape': 27.432963038172094, 'eval_runtime': 28.961, 'eval_samples_per_second': 4.869, 'eval_steps_per_second': 1.243, 'epoch': 3.0}
{'train_runtime': 252.6737, 'train_samples_per_second': 6.661, 'train_steps_per_second': 0.843, 'train_loss': 0.512459804194634, 'epoch': 3.0}


TrainOutput(global_step=213, training_loss=0.512459804194634, metrics={'train_runtime': 252.6737, 'train_samples_per_second': 6.661, 'train_steps_per_second': 0.843, 'total_flos': 118200384479232.0, 'train_loss': 0.512459804194634, 'epoch': 3.0})

In [10]:
# Save the trained model and tokenizer
model.save_pretrained('../models/t5')
tokenizer.save_pretrained('../models/t5')


('../models/t5\\tokenizer_config.json',
 '../models/t5\\special_tokens_map.json',
 '../models/t5\\spiece.model',
 '../models/t5\\added_tokens.json')